In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
import cv2
import os
import numpy as np
import gc
from sklearn.model_selection import train_test_split
from tensorflow.keras.layers import Dropout
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D
from tensorflow.keras.layers import Conv2D, AveragePooling2D, Flatten, Dense
from tensorflow.keras.models import Sequential
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from keras.utils import to_categorical
import random
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, AveragePooling2D, Flatten, Dense
from tensorflow.keras.regularizers import l2
from keras.utils import to_categorical
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, AveragePooling2D, Flatten, Dense
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.regularizers import l2

In [ ]:
model = Sequential()
model.add(Conv2D(6, kernel_size=(5, 5), strides=(1, 1), activation='tanh', input_shape=(32,32, 3)))
model.add(AveragePooling2D(pool_size=(2, 2), strides=(2, 2)))
model.add(Conv2D(16, kernel_size=(5, 5), strides=(1, 1), activation='tanh'))
model.add(AveragePooling2D(pool_size=(2, 2), strides=(2, 2)))
model.add(Flatten())
model.add(Dense(120, activation='tanh', kernel_regularizer=l2(0.01)))
model.add(Dense(84, activation='tanh', kernel_regularizer=l2(0.01)))
model.add(Dense(7, activation='softmax'))

In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, AveragePooling2D, Flatten, Dense, BatchNormalization
from tensorflow.keras.regularizers import l2

model = Sequential()
model.add(Conv2D(6, kernel_size=(5, 5), strides=(1, 1), activation='tanh', input_shape=(8,8, 3)))
model.add(BatchNormalization())
model.add(AveragePooling2D(pool_size=(2, 2), strides=(2, 2)))
model.add(Conv2D(16, kernel_size=(3, 3), strides=(1, 1), activation='tanh', padding='same'))
model.add(BatchNormalization())
model.add(AveragePooling2D(pool_size=(2, 2), strides=(2, 2)))
model.add(Flatten())
model.add(Dense(120, activation='tanh', kernel_regularizer=l2(0.01)))
model.add(BatchNormalization())
model.add(Dense(84, activation='tanh', kernel_regularizer=l2(0.01)))
model.add(BatchNormalization())
model.add(Dense(7, activation='softmax'))


In [ ]:
import cv2
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense

def merge_data(video, text, batch_size=32, resize_shape=(32,32)):
    data = []
    while True:
        frames = []
        frame_nums = []
        for i in range(batch_size):
            ret, frame = video.read()
            if not ret:
                break
            frame_num = int(video.get(cv2.CAP_PROP_POS_FRAMES))
            frame = cv2.resize(frame, resize_shape)
            frames.append(frame)
            frame_nums.append(frame_num)
        if len(frames) == 0:
            break
        instrument_data = text[text['Frame'].isin(frame_nums)]
        for i in range(len(frames)):
            frame = frames[i]
            frame_num = frame_nums[i]
            instrument_info = instrument_data[instrument_data['Frame'] == frame_num]
            if len(instrument_info) == 0:
                instrument_info = pd.DataFrame([[frame_num, 0, 0, 0, 0, 0, 0, 0]], columns=text.columns)
            data.append((frame, instrument_info))
    video.release()
    return data


In [ ]:
from sklearn.utils import resample

def preprocess_data(data):
    X = []
    y = []
    for frame, instrument_data in data:
        frame = frame / 255.0
        X.append(frame)
        y.append(instrument_data.values[0][1:])
    X = np.array(X)
    y = np.array(y)
    df = pd.DataFrame(np.concatenate((X.reshape(X.shape[0], -1), y), axis=1),
                      columns=['pixel{}'.format(i) for i in range(X.shape[1]*X.shape[2]*X.shape[3])] + ['label{}'.format(i) for i in range(y.shape[1])])
    majority = df[df['label0']==1]
    minority = df[df['label0']==0]

    minority_oversampled = resample(minority,
                                    replace=True,
                                    n_samples=len(majority),
                                    random_state=123)

    df_oversampled = pd.concat([majority, minority_oversampled])

    X_oversampled = df_oversampled.iloc[:, :X.shape[1]*X.shape[2]*X.shape[3]].values.reshape(-1, X.shape[1], X.shape[2], X.shape[3])
    y_oversampled = df_oversampled.iloc[:, X.shape[1]*X.shape[2]*X.shape[3]:].values
    
    datagen = ImageDataGenerator(rotation_range=20, 
                                 width_shift_range=0.2, 
                                 height_shift_range=0.2, 
                                 shear_range=0.2, 
                                 zoom_range=0.2,
                                 horizontal_flip=True, 
                                 fill_mode='nearest')

    X_aug = []
    y_aug = []
    for i in range(len(X_oversampled)):
        x = X_oversampled[i]
        y_val = y_oversampled[i]
        x = np.expand_dims(x, axis=0)
        y_val = np.expand_dims(y_val, axis=0)
        for batch in datagen.flow(x, y_val, batch_size=1):
            X_aug.append(batch[0][0])
            y_aug.append(batch[1][0])
            break
    X_aug = np.array(X_aug)
    y_aug = np.array(y_aug)
    
    X_oversampled = np.concatenate((X_oversampled, X_aug), axis=0)
    y_oversampled = np.concatenate((y_oversampled, y_aug), axis=0)

    return X_oversampled, y_oversampled


In [ ]:
def create_model(input_shape):
    model = Sequential()
    model.add(Conv2D(16, kernel_size=(5, 5), strides=(1, 1), activation='tanh', input_shape=input_shape))
    model.add(Dropout(0.25))
    model.add(AveragePooling2D(pool_size=(2, 2), strides=(2, 2)))
    model.add(Conv2D(32, kernel_size=(5, 5), strides=(1, 1), activation='tanh'))
    model.add(Dropout(0.5))
    model.add(AveragePooling2D(pool_size=(2, 2), strides=(2, 2)))
    model.add(Flatten())
    model.add(Dense(120, activation='tanh', kernel_regularizer=l2(0.01)))
    model.add(Dense(84, activation='tanh', kernel_regularizer=l2(0.01)))
    model.add(Dense(7, activation='softmax'))
    
    model.compile(optimizer='SGD',
                  loss='categorical_crossentropy',
                  metrics=['accuracy'])
    return model


In [ ]:
from keras.layers import BatchNormalization


In [ ]:
def create_model(input_shape):
    model = Sequential()
    model.add(Conv2D(96, kernel_size=(7, 7), strides=(2, 2), activation='relu', input_shape=input_shape, padding='same'))
    model.add(MaxPooling2D(pool_size=(3, 3), strides=(2, 2), padding='same'))
    model.add(BatchNormalization())
    
    model.add(Conv2D(256, kernel_size=(5, 5), strides=(2, 2), activation='relu', padding='same'))
    model.add(MaxPooling2D(pool_size=(3, 3), strides=(2, 2), padding='same'))
    model.add(BatchNormalization())
    
    model.add(Conv2D(384, kernel_size=(3, 3), activation='relu', padding='same'))
    model.add(Conv2D(384, kernel_size=(3, 3), activation='relu', padding='same'))
    model.add(Conv2D(256, kernel_size=(3, 3), activation='relu', padding='same'))
    model.add(MaxPooling2D(pool_size=(3, 3), strides=(2, 2), padding='same'))
    model.add(BatchNormalization())
    
    model.add(Flatten())
    model.add(Dense(4096, activation='relu'))
    model.add(Dropout(0.5))
    model.add(Dense(4096, activation='relu'))
    model.add(Dropout(0.5))
    model.add(Dense(7, activation='softmax'))
    
    model.compile(optimizer='adam',
                  loss='categorical_crossentropy',
                  metrics=['accuracy'])
    
    return model


In [ ]:
def create_model(input_shape):
    model = Sequential()
    
    model.add(Conv2D(32, kernel_size=(3, 3), activation='relu', input_shape=input_shape))
    model.add(MaxPooling2D(pool_size=(2, 2)))
    
    model.add(Conv2D(64, kernel_size=(3, 3), activation='relu'))
    model.add(MaxPooling2D(pool_size=(2, 2)))
    
    model.add(Conv2D(128, kernel_size=(3, 3), activation='relu'))
    model.add(MaxPooling2D(pool_size=(2, 2)))
    
    model.add(Flatten())
    model.add(Dense(128, activation='relu'))
    model.add(Dense(7, activation='softmax'))
    
    model.compile(optimizer='adam',
                  loss='categorical_crossentropy',
                  metrics=['accuracy'])
    
    return model


In [ ]:
import cv2
import pandas as pd
video1 = cv2.VideoCapture('/kaggle/input/d/sahithivunnava/surgical/train/tool_video_04.mp4')
video2 = cv2.VideoCapture('/kaggle/input/d/sahithivunnava/surgical/11/tool_video_07.mp4')
video3 = cv2.VideoCapture('/kaggle/input/d/sahithivunnava/surgical/test/tool_video_15.mp4')
text1 = pd.read_csv('/kaggle/input/d/sahithivunnava/surgical/train/tool_video_04.txt',sep='\t')
text2 = pd.read_csv('/kaggle/input/d/sahithivunnava/surgical/11/tool_video_07.txt',sep='\t')
text3 = pd.read_csv('/kaggle/input/d/sahithivunnava/surgical/test/tool_video_15.txt',sep='\t')
text1.columns = ['Frame', 'Grasper', 'Bipolar', 'Hook', 'Scissors', 'Clipper', 'Irrigator', 'SpecimenBag']
text2.columns = ['Frame', 'Grasper', 'Bipolar', 'Hook', 'Scissors', 'Clipper', 'Irrigator', 'SpecimenBag']
text3.columns = ['Frame', 'Grasper', 'Bipolar', 'Hook', 'Scissors', 'Clipper', 'Irrigator', 'SpecimenBag']


In [ ]:
data1 = merge_data(video1, text1)
data2 = merge_data(video2, text2)
data3 = merge_data(video3, text3)

In [ ]:
X1, y1 = preprocess_data(data1)
X2, y2 = preprocess_data(data2)
X3, y3 = preprocess_data(data3)

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(np.concatenate([X1, X2, X3]), 
                                                    np.concatenate([y1, y2, y3]), 
                                                    test_size=0.2, 
                                                    random_state=42)

In [ ]:
model = create_model(input_shape=X_train[0].shape)
# from sklearn.utils.class_weight import compute_class_weight

# class_weights = compute_class_weight('balanced', np.unique(y_train), y_train)
# class_weights_dict = dict(enumerate(class_weights))
model.fit(X_train, y_train, epochs=20, batch_size=32, validation_data=(X_test, y_test))

In [ ]:
score = model.evaluate(X_test, y_test)
print(f'Test loss: {score[0]}')
print(f'Test accuracy: {score[1] * 100}')

In [ ]:
y_pred = model.predict(X_test)

In [ ]:
from sklearn.metrics import classification_report

In [ ]:
y_pred = model.predict(X_test)
y_pred_labels = np.argmax(y_pred, axis=1)
y_pred_one_hot = to_categorical(y_pred_labels, num_classes=7)
print(classification_report(y_test, y_pred_one_hot))


In [ ]:
# # Calculate accuracy for each instrument class
# classes = ['Grasper', 'Bipolar', 'Hook', 'Scissors', 'Clipper', 'Irrigator', 'SpecimenBag']
# for i, cls in enumerate(classes):
#     indices = np.where(y_true == i)[0]
#     class_accuracy = np.mean(np.equal(np.argmax(np.expand_dims(y_pred[indices], axis=0), axis=1), y_true[indices]))
#     print(f'Accuracy for {cls}: {class_accuracy:.3f}')
